In [1]:
# libraries used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.svm import LinearSVC, SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, StackingClassifier

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.neighbors import KNeighborsClassifier


from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
import matplotlib.patches as patches
from aif360.algorithms.preprocessing import Reweighing
#from packages import *
#from ml_fairness import *
import matplotlib.pyplot as plt
import seaborn as sns



from IPython.display import Markdown, display

import ipynbname
nb_fname = ipynbname.name()
nb_path = ipynbname.path()

import pickle

In [2]:
# This DataFrame is created to stock differents models and fair metrics that we produce in this notebook
algo_metrics = pd.DataFrame(columns=['model', 'fair_metrics', 'prediction', 'probs'])

def add_to_df_algo_metrics(algo_metrics, model, fair_metrics, preds, probs, name):
    return algo_metrics.append(pd.DataFrame(data=[[model, fair_metrics, preds, probs]], columns=['model', 'fair_metrics', 'prediction', 'probs'], index=[name]))

In [3]:
def fair_metrics(dataset, pred, pred_is_dataset=False):
    if pred_is_dataset:
        dataset_pred = pred
    else:
        dataset_pred = dataset.copy()
        dataset_pred.labels = pred
    
    cols = ['statistical_parity_difference', 'equal_opportunity_difference', 'average_abs_odds_difference',  'disparate_impact', 'theil_index']
    obj_fairness = [[0,0,0,1,0]]
    
    fair_metrics = pd.DataFrame(data=obj_fairness, index=['objective'], columns=cols)
    
    for attr in dataset_pred.protected_attribute_names:
        idx = dataset_pred.protected_attribute_names.index(attr)
        privileged_groups =  [{attr:dataset_pred.privileged_protected_attributes[idx][0]}] 
        unprivileged_groups = [{attr:dataset_pred.unprivileged_protected_attributes[idx][0]}] 
        
        classified_metric = ClassificationMetric(dataset, 
                                                     dataset_pred,
                                                     unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)

        metric_pred = BinaryLabelDatasetMetric(dataset_pred,
                                                     unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)

        acc = classified_metric.accuracy()

        row = pd.DataFrame([[metric_pred.mean_difference(),
                                classified_metric.equal_opportunity_difference(),
                                classified_metric.average_abs_odds_difference(),
                                metric_pred.disparate_impact(),
                                classified_metric.theil_index()]],
                           columns  = cols,
                           index = [attr]
                          )
        fair_metrics = fair_metrics.append(row)    
    
    fair_metrics = fair_metrics.replace([-np.inf, np.inf], 2)
        
    return fair_metrics

def plot_fair_metrics(fair_metrics):
    fig, ax = plt.subplots(figsize=(20,4), ncols=5, nrows=1)

    plt.subplots_adjust(
        left    =  0.125, 
        bottom  =  0.1, 
        right   =  0.9, 
        top     =  0.9, 
        wspace  =  .5, 
        hspace  =  1.1
    )

    y_title_margin = 1.2

    plt.suptitle("Fairness metrics", y = 1.09, fontsize=20)
    sns.set(style="dark")

    cols = fair_metrics.columns.values
    obj = fair_metrics.loc['objective']
    size_rect = [0.2,0.2,0.2,0.4,0.25]
    rect = [-0.1,-0.1,-0.1,0.8,0]
    bottom = [-1,-1,-1,0,0]
    top = [1,1,1,2,1]
    bound = [[-0.1,0.1],[-0.1,0.1],[-0.1,0.1],[0.8,1.2],[0,0.25]]

    display(Markdown("### Check bias metrics :"))
    display(Markdown("A model can be considered bias if just one of these five metrics show that this model is biased."))
    for attr in fair_metrics.index[1:len(fair_metrics)].values:
        display(Markdown("#### For the %s attribute :"%attr))
        check = [bound[i][0] < fair_metrics.loc[attr][i] < bound[i][1] for i in range(0,5)]
        display(Markdown("With default thresholds, bias against unprivileged group detected in **%d** out of 5 metrics"%(5 - sum(check))))

    for i in range(0,5):
        plt.subplot(1, 5, i+1)
        ax = sns.barplot(x=fair_metrics.index[1:len(fair_metrics)], y=fair_metrics.iloc[1:len(fair_metrics)][cols[i]])
        
        for j in range(0,len(fair_metrics)-1):
            a, val = ax.patches[j], fair_metrics.iloc[j+1][cols[i]]
            marg = -0.2 if val < 0 else 0.1
            ax.text(a.get_x()+a.get_width()/5, a.get_y()+a.get_height()+marg, round(val, 3), fontsize=15,color='black')

        plt.ylim(bottom[i], top[i])
        plt.setp(ax.patches, linewidth=0)
        ax.add_patch(patches.Rectangle((-5,rect[i]), 10, size_rect[i], alpha=0.3, facecolor="green", linewidth=1, linestyle='solid'))
        plt.axhline(obj[i], color='black', alpha=0.3)
        plt.title(cols[i])
        ax.set_ylabel('')    
        ax.set_xlabel('')

In [4]:
def get_fair_metrics_and_plot(data, model, plot=False, model_aif=False):
    pred = model.predict(data).labels if model_aif else model.predict(data.features)
    # fair_metrics function available in the metrics.py file
    fair = fair_metrics(data, pred)

    if plot:
        # plot_fair_metrics function available in the visualisations.py file
        # The visualisation of this function is inspired by the dashboard on the demo of IBM aif360 
        plot_fair_metrics(fair)
        display(fair)
    
    return fair

In [5]:
import pandas as pd
import os
import glob

i = 0
# use glob to get all the csv files 
# in the folder
TM_dict = dict()
paths = ["..\\..\\Titanic\\Results\\VotingClassifier\\"]
for path in paths:
    csv_files = glob.glob(os.path.join(path, "*.pkl"))
    for f in csv_files:
        TM_dict[f] = pd.read_pickle(f)
        

AC_dict = dict()    
paths = ["..\\..\\AdultNotebook\\Results\\VotingClassifier\\"]
for path in paths:
    csv_files = glob.glob(os.path.join(path, "*.pkl"))

    for f in csv_files:
        AC_dict[f] = pd.read_pickle(f)
        

BM_dict = dict()
paths = ["..\\..\\BankMarketingNotebook\\Results\\VotingClassifier\\"]
for path in paths:
    csv_files = glob.glob(os.path.join(path, "*.pkl"))
    for f in csv_files:
        BM_dict[f] = pd.read_pickle(f)
        
GC_dict = dict()
paths = ["..\\..\\GermanCredit\\Results\\VotingClassifier\\"]
for path in paths:
    print(path)
    csv_files = glob.glob(os.path.join(path, "*.pkl"))
    for f in csv_files:
        GC_dict[f] = pd.read_pickle(f)
    
    
models = [TM_dict, AC_dict, BM_dict, GC_dict]

..\..\GermanCredit\Results\VotingClassifier\


In [38]:
AC_dict.keys()

dict_keys(['..\\..\\AdultNotebook\\Results\\VotingClassifier\\0-income-prediction-84-369-accuracy.pkl', '..\\..\\AdultNotebook\\Results\\VotingClassifier\\0-income-prediction-84-369-accuracy_Test.pkl', '..\\..\\AdultNotebook\\Results\\VotingClassifier\\0-income-prediction-84-369-accuracy_Train.pkl', '..\\..\\AdultNotebook\\Results\\VotingClassifier\\1-eda-and-income-predictions-86-78-accuracy.pkl', '..\\..\\AdultNotebook\\Results\\VotingClassifier\\1-eda-and-income-predictions-86-78-accuracy_Test.pkl', '..\\..\\AdultNotebook\\Results\\VotingClassifier\\1-eda-and-income-predictions-86-78-accuracy_Train.pkl', '..\\..\\AdultNotebook\\Results\\VotingClassifier\\2-income-census-prediction-gradient-boosting-algos.pkl', '..\\..\\AdultNotebook\\Results\\VotingClassifier\\2-income-census-prediction-gradient-boosting-algos_Test.pkl', '..\\..\\AdultNotebook\\Results\\VotingClassifier\\2-income-census-prediction-gradient-boosting-algos_Train.pkl', '..\\..\\AdultNotebook\\Results\\VotingClassifier\

In [32]:
import ipynbname
nb_fname = ipynbname.name()
nb_path = ipynbname.path()

Overall_metrics = []

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
from csv import writer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

for dataset in models:
    
    for i in range (0, len(dataset.keys()),3):
        print(i)
        data_orig_train = pd.read_pickle(list(dataset.keys())[i+2])
        data_orig_test = pd.read_pickle(list(dataset.keys())[i+1])
        model = pd.read_pickle(list(dataset.keys())[i])
        model.set_params(**{'voting': 'soft'})
        print(model.voting)
        
        X_train = data_orig_train.features
        y_train = data_orig_train.labels.ravel()

        X_test = data_orig_test.features
        y_test = data_orig_test.labels.ravel()
        
        
        final_metrics = []
        accuracy = []
        f1= []

        
        mdl = model.fit(X_train, y_train)
        try:
            yy = mdl.predict(X_test)
        except Exception:
            continue
        accuracy.append(accuracy_score(y_test, yy))
        f1.append(f1_score(y_test, yy))
        fair = get_fair_metrics_and_plot(data_orig_test, mdl)                           
        fair_list = fair.iloc[1].tolist()
        fair_list[3] = np.log(fair_list[3])
        final_metrics.append(fair_list)

        Overall_metrics.append(list(sum(x) for x in final_metrics))

0
soft
3
soft
6
soft


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.


9
soft
12
soft
15
soft
0
soft
3
soft
[15:20:17] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.


6
soft
9
soft


...

15
soft
0
soft
3
soft


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


6
soft
9
soft
12
soft


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


15
soft


In [33]:
Overall_metrics

[[-2.950100897595755],
 [-3.069187238539814],
 [-2.5649280737829283],
 [-2.7289290890882674],
 [-2.742688783983291],
 [-2.383969668414516],
 [-1.1136321583993403],
 [-2.200685870428438],
 [-1.2700246545327036],
 [-1.442539325925548],
 [-1.4113798335785939],
 [-1.3043910501196991],
 [1.454318962403024],
 [1.6547155205779986],
 [0.7766088056544911],
 [0.08810634989978808],
 [1.5364375712287035],
 [0.7514995298969995],
 [0.021425591380663606],
 [-0.3156891524534373],
 [-0.08275798175722539],
 [-0.06965252217381004],
 [-0.13856655568922915]]

In [34]:
import csv

with open("SoftVoting.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(Overall_metrics)